In [ ]:
#requirements.txt
functions-framework==3.*
google-api-python-client
google-auth-httplib2
google-auth-oauthlib
google-cloud-secret-manager==2.1.0

google-cloud-aiplatform>=1.4.2
google-cloud-aiplatform[prediction]>=1.16.0
google-generativeai
google-cloud-vision
google-cloud-language==2.12.0
google-cloud-speech
SpeechRecognition
google-cloud-storage

opencv-python==4.8.0.76
pandas==1.5.3
numpy==1.23.5
spacy==3.7.2
pt-core-news-lg @ https://github.com/explosion/spacy-models/releases/download/pt_core_news_lg-3.7.0/pt_core_news_lg-3.7.0-py3-none-any.whl
en_core_web_lg @ https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl
nltk==3.8.1
ffmpeg-python==0.2.0
moviepy==1.0.3


In [ ]:
import functions_framework
from google.cloud import secretmanager

import io
import os
import math
import numpy as np
import pandas as pd
import pytz

import vertexai
from vertexai.language_models import TextGenerationModel
import google.generativeai as genai
import google.auth
import spacy
import nltk
import cv2
import ffmpeg
import moviepy.editor as mp

from google.auth import default
from google.oauth2 import service_account

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload

from google.cloud import aiplatform
from google.cloud import storage
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from google.api_core.client_options import ClientOptions
from google.cloud import language_v2
from google.cloud import vision

from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
from datetime import timedelta
from datetime import datetime
from nltk.tokenize import word_tokenize, sent_tokenize
set(pytz.all_timezones_set)

def get_google_sheet(SPREADSHEET_ID, RANGE_NAME):


  sheet = ''
  df = ''
  header = ''

  try:
    creds, _ = default()
    client_service_sheet_api = build("sheets", "v4", credentials=creds)
    service = client_service_sheet_api

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME)
        .execute()
    )

    values = result.get("values", [])
    if not values:
      print("No data found.")
      return

    header = result.get('values', [])[0]   # Assumes first line is header!
    values = result.get('values', [])[1:]  # Everything else is data.
    all_data = []

    for col_id, col_name in enumerate(header):

        column_data = []
        for row in values:
            column_data.append(row[col_id])
        ds = pd.Series(data=column_data, name=col_name)
        all_data.append(ds)
    df = pd.concat(all_data, axis=1)

  except HttpError as err:
    print(err)

  return sheet, df, header

def download_file(real_file_id):

  creds, _ = google.auth.default()

  try:
    client_service_drive_api = build("drive", "v3", credentials=creds)
    service = client_service_drive_api
    file_id = real_file_id
    request = service.files().get_media(fileId=file_id)

    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)

    done = False
    while done is False:
      status, done = downloader.next_chunk()
      print(f"Download {int(status.progress() * 100)}.")

  except HttpError as error:
    print(f"An error occurred: {error}")
    file = None

  return file.getvalue()

def upload_to_bucket(client_storage_api, blob_name, path_to_file, bucket_name):

    storage_client = client_storage_api

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

def transcribe_batch_gcs_input_inline_output_v4(
    client_speech_api,
    project_id: str,
    gcs_uri_upload: str,
    gcs_uri_download: str,
    audio_input,
    language
) -> cloud_speech.BatchRecognizeResults:

    long_audio_uri = f"{gcs_uri_upload}/audio/{audio_input}"


    lang_code = ""
    if (language == "Portuguese"):
        lang_code = "pt-BR"
    else:
        lang_code = "en-US"


    recognizer_id = f'chirp-{lang_code.lower()}'

    # Instantiates a client
    client = client_speech_api

    config = cloud_speech.RecognitionConfig(
        features = cloud_speech.RecognitionFeatures(
            enable_automatic_punctuation=True,
            enable_word_time_offsets=True
        ),
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=[lang_code],
        model="chirp",
      )


    file_metadata = cloud_speech.BatchRecognizeFileMetadata(uri=gcs_uri_upload)

    request = cloud_speech.BatchRecognizeRequest(
        recognizer=f"projects/{project_id}/locations/us-central1/recognizers/{recognizer_id}", #
        config=config,
        files=[file_metadata],
        recognition_output_config=cloud_speech.RecognitionOutputConfig(
            inline_response_config=cloud_speech.InlineOutputConfig(),
        ),
    )
    response = client.batch_recognize(request=request)

    print("Waiting for operation to complete... ")
    operation = response.result(timeout=1800)

    print("Transcription completed.")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    transcript = []
    words_data = {'word': [], 'start_offset': [], 'end_offset': []}
    words_offset = pd.DataFrame(words_data)
    index_alternative_words = 0
    if (len(operation.results[gcs_uri_upload].transcript.results) > 0):
        print("Transcription loaded.")
    else:
        print("It was not possible to load transcription.")

    full_transcription = ""
    audio_input_duration = 0
    full_results = operation.results
    for result in operation.results[gcs_uri_upload].transcript.results:
        if (len(result.alternatives) > 0):

            transcript.append(result.alternatives[0].transcript)
            alternative = result.alternatives[0]
            full_transcription += result.alternatives[0].transcript
            audio_input_duration = result.result_end_offset.seconds + result.result_end_offset.microseconds / 1e9

            for word_info in alternative.words:

                word_with_punktuation = word_info.word
                words_with_punktuation = word_tokenize(word_with_punktuation)
                words_with_punktuation=[word.lower() for word in words_with_punktuation if word.isalpha()]
                for word in words_with_punktuation:
                    start_offset = word_info.start_offset
                    end_offset = word_info.end_offset
                    words_offset.loc[index_alternative_words,'word'] = word
                    words_offset.loc[index_alternative_words,'start_offset'] = start_offset.total_seconds()
                    words_offset.loc[index_alternative_words,'end_offset'] = end_offset.total_seconds()
                index_alternative_words += 1

    return transcript, words_offset, full_transcription, audio_input_duration, full_results

def generate_srt(full_results, gcs_uri_upload, filename, subtitles_sentence_offset):
    subtitles = ""
    index = 1

    for result in full_results[gcs_uri_upload].transcript.results:

        if (len(result.alternatives) > 0) and (len(result.alternatives[0].words) > 0):
            idx_word = 0
            count_word = 0
            line = ""
            start_time_str = ''
            end_time_str = ''
            start_time = 0
            start_seconds = 0
            for word in result.alternatives[0].words:
                line = line + str(word.word) + " "
                if (count_word == 0):
                    start_time = word.start_offset
                    start_seconds = start_time.seconds + start_time.microseconds * 1e-9
                    start_time_str = format_time(start_seconds)
                    count_word += 1
                elif (((idx_word+1) == len(result.alternatives[0].words)) or (count_word == 15)):
                    end_time = word.end_offset
                    end_seconds = end_time.seconds + end_time.microseconds * 1e-9
                    end_time_str = format_time(end_seconds)
                    subtitles += f"{index}\n{start_time_str} --> {end_time_str}\n{line}\n\n"
                    subtitles_sentence_offset.loc[len(subtitles_sentence_offset)] = [str(line), int(start_seconds), int(end_seconds), '', '']
                    index += 1
                    count_word = 0
                    line = ''
                else:
                    count_word += 1
                idx_word += 1


    with open(filename, 'w') as srt_file:
        srt_file.write(subtitles)

    return subtitles_sentence_offset

def format_time(seconds):
    time_obj = timedelta(seconds=seconds)
    return str(time_obj).replace('.', ',')

def get_sus_sentences_from_ai_gemini(
    full_transcript,
    language,
    filename
) -> str:

    text_to_separate = full_transcript
    prompt_en = f"While you neither invent words nor translate, and stay strict to the original text without removing the words from that text, separate the following text into sentences with no more than 32 tokens:\n{text_to_separate}"
    prompt_pt = f"Embora você não invente palavras nem traduza e permaneça rigoroso com o texto original sem remover as palavras desse texto, separe o texto a seguir em frases com no máximo 32 tokens:\n{text_to_separate}"
    if (language=='Portuguese'):
        prompt=prompt_pt
    else:
        prompt=prompt_en

    model = genai.GenerativeModel('gemini-pro')

    responses = model.generate_content(
            prompt,
            generation_config={
            "temperature": 0.2,
            "max_output_tokens": 16384,
            "top_p": 0.2,
            "top_k": 1,
        },
        stream=False
        )

    response_ai_gemini = ''
    for response in responses:
        response_ai_gemini += response.text + '\n'

    with open(filename, 'w') as srt_file:
        srt_file.write(response_ai_gemini)

    return response_ai_gemini

def analyze_sentiment(client_natural_language_api, text_content: str, language):

    client = client_natural_language_api
    document_type_in_plain_text = language_v2.Document.Type.PLAIN_TEXT

    if (language == "Portuguese"):
        lang_code = "pt"
    else:
        lang_code = "en"

    text_content = text_content.strip()
    document = {
        "content": text_content,
        "type_": document_type_in_plain_text,
        "language_code": lang_code,
    }

    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    sentence_sentiment_score = 0
    sentence_sentiment_magnitude = 0

    if (response != ''):
        sentence_sentiment_score = response.document_sentiment.score
        sentence_sentiment_magnitude = response.document_sentiment.magnitude

    return sentence_sentiment_score, sentence_sentiment_magnitude

def analyze_sentiment_document(client_natural_language_api, text_content: str, language, document_sentences_offset):

    client = client_natural_language_api
    document_type_in_plain_text = language_v2.Document.Type.PLAIN_TEXT

    if (language == "Portuguese"):
        lang_code = "pt"
    else:
        lang_code = "en"

    text_content = text_content.strip()
    document = {
        "content": text_content,
        "type_": document_type_in_plain_text,
        "language_code": lang_code,
    }

    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    document_sentence = ''
    document_sentence_sentiment = ''
    document_sentence_sentiment_score = 0
    document_sentence_sentiment_magnitude = 0

    if (response != ''):

        for document_sentence_response in response.sentences:
            document_sentence = document_sentence_response.text.content
            document_sentence_sentiment_score = document_sentence_response.sentiment.score
            document_sentence_sentiment_magnitude = document_sentence_response.sentiment.magnitude

            document_sentence_sentiment = 'Neutral'
            if (document_sentence_sentiment_magnitude >= 0.25):
                if (document_sentence_sentiment_score > 0.25):
                    document_sentence_sentiment = 'Positive'
                if (document_sentence_sentiment_score < -0.25):
                    document_sentence_sentiment = 'Negative'
            document_sentences_offset.loc[len(document_sentences_offset)] = [document_sentence, -1, -1, document_sentence_sentiment, document_sentence_sentiment_magnitude]


    return document_sentences_offset

def upload_basic(name, file_type, folder_id, evaluated_video_id):

  creds, _ = google.auth.default()

  try:
    # create drive api client
    client_service_drive_api = build("drive", "v3", credentials=creds)
    service = client_service_drive_api
    file_metadata = {"name": name, "parents": [folder_id]}
    media = MediaFileUpload(name, mimetype=file_type)

    if (evaluated_video_id == ''):
        file = (
            service.files()
            .create(body=file_metadata, media_body=media, fields="id")
            .execute()
        )
    else:
        file = (
            service.files()
            .update(fileId=evaluated_video_id, media_body=media)
            .execute()
        )
    print(f'File ID: {file.get("id")}')

  except HttpError as error:
    print(f"An error occurred: {error}")
    file = None

  return file.get("id")

def compare_sentence_similarity(sentence1, sentence2, nlp):

  # Process the sentences using spaCy
  doc1 = nlp(sentence1)
  doc2 = nlp(sentence2)

  # Create a list of word vectors for each sentence
  sentence_vectors = [token.vector for token in doc1 if token.has_vector], [token.vector for token in doc2 if token.has_vector]

  # Check if both sentences have at least one vector
  if not all(sentence_vectors):
    return 0.0

  similarity_score = doc1.similarity(doc2)

  return similarity_score

def gsheet_update_values(spreadsheet_id, range_name, value_input_option, values):

  creds, _ = google.auth.default()

  try:
    client_service_sheet_api = build("sheets", "v4", credentials=creds)
    service = client_service_sheet_api
    body = {"values": values}
    result = (
        service.spreadsheets()
        .values()
        .update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption=value_input_option,
            body=body,
        )
        .execute()
    )
    print(f"{result.get('updatedCells')} cells updated.")

    return result

  except HttpError as error:
    print(f"An error occurred: {error}")

    return error

def share_file(real_file_id, real_user, real_domain):

  creds, _ = google.auth.default()

  try:
    client_service_drive_api = build("drive", "v3", credentials=creds)
    service = client_service_drive_api
    ids = []
    file_id = real_file_id

    def callback(request_id, response, exception):
      if exception:
        # Handle error
        print(exception)
      else:
        print(f"Request_Id: {request_id}")
        print(f'Permission Id: {response.get("id")}')
        ids.append(response.get("id"))

    batch = service.new_batch_http_request(callback=callback)
    user_permission = {
        "type": "user",
        "role": "reader",
        "emailAddress": real_user,
    }
    batch.add(
        service.permissions().create(
            fileId=file_id,
            body=user_permission,
            fields="id",
        )
    )
    domain_permission = {
        "type": "domain",
        "role": "reader",
        "domain": real_domain,
    }
    batch.add(
        service.permissions().create(
            fileId=file_id,
            body=domain_permission,
            fields="id",
        )
    )
    batch.execute()

  except HttpError as error:
    print(f"An error occurred: {error}")
    ids = None

  return ids


def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    generation_match_precondition = None

    blob.reload()
    generation_match_precondition = blob.generation

    blob.delete(if_generation_match=generation_match_precondition)

    print(f"Blob {blob_name} deleted.")

############
@functions_framework.http
def uxapp_engine_http(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    request_json = request.get_json(silent=True)
    if request_json and 'product' in request_json:
        product = request_json['product']
    else:
        product = ''
    if request_json and 'tasktime' in request_json:
        tasktime = request_json['tasktime']
    else:
        tasktime = ''

    if ((product != '') and (tasktime != '')):

        print("Working on the enviromemt... ")
        tz = pytz.timezone('America/Sao_Paulo')
        now = datetime.now(tz=tz)
        current_time = now.strftime("%H:%M:%S")
        job_start_time = current_time
        print("Current Time =", job_start_time)

        nltk.download('punkt')

        SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

        CREDENTIALS_LOCATION = "" #change
        os.environ["CREDENTIALS"] = CREDENTIALS_LOCATION
        SERVICE_ACCOUNT_FILE = os.environ["CREDENTIALS"]
        creds = service_account.Credentials.from_service_account_file(
              SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        creds, _ = default()

        print("Starting Google Cloud clients... ")

        client_natural_language_api = language_v2.LanguageServiceClient()
        client_speech_api = SpeechClient(client_options=ClientOptions(
                api_endpoint=f'us-central1-speech.googleapis.com'))
        client_storage_api = storage.Client()
        client_service_drive_api = build("drive", "v3", credentials=creds)
        client_service_sheet_api = build("sheets", "v4", credentials=creds)
        project = "" #change
        location = "" #change
        vertexai.init(project=project, location=location)
        client_vision_api = vision.ImageAnnotatorClient()

        ############
        print("Retriving data... ")

        # The ID and range of a sample spreadsheet.
        SPREADSHEET_ID = "" #change

        RANGE_NAME_SPREADSHEET_PARTICIPANT = "Participant"
        gsheet, participant_gsheet, participant_header = get_google_sheet(SPREADSHEET_ID, RANGE_NAME_SPREADSHEET_PARTICIPANT)
        original_data_participant = participant_gsheet.copy()

        RANGE_NAME_SPREADSHEET_EXPERIMENT = "Experiment"
        gsheet, original_data_experiment_gsheet, SPREADSHEET_HEADER = get_google_sheet(SPREADSHEET_ID, RANGE_NAME_SPREADSHEET_EXPERIMENT)
        original_data_experiment = original_data_experiment_gsheet.copy()

        ######
        print("Setting status... ")

        original_data_experiment = original_data_experiment.reset_index(drop=True)
        index_experiment = 0
        count_analysis = 0
        RANGE_NAME = ''
        for idx1_original_data_experiment, row in original_data_experiment.iterrows():
            if ((str(row['Product']) == product) and (str(row['Task Start Time']) == tasktime) and (str(row['Status Script']) == 'In Analysis')):

                index_experiment = idx1_original_data_experiment
                experiment_id = str(row['Participant']) + ' - ' + str(row['Task Objective'])
                video_input = str(row['Participant']) + ' - ' + str(row['Task Objective']) + '.mp4'
                original_video_url = str(row['Record Link'])
                evaluated_video_url = str(row['Evaluated Link'])
                evaluated_audio_url = str(row['Evaluated Audio Link'])
                evaluated_subtitle_url = str(row['Evaluated Subtitle Link'])
                evaluated_ai_sus_sentences_url = str(row['Evaluated AI SUS Sentences Link'])
                participant_id = str(row['Participant'])
                RANGE_NAME = RANGE_NAME_SPREADSHEET_EXPERIMENT+'!A' + str(index_experiment+2) + ':' + 'AY' + str(index_experiment+2)
                row_original_data_experiment = row
                count_analysis = 1
        if (count_analysis == 0):
            print("No experiment to analyze found")
            return 'Nothing to do. Product - Task time: {}!'.format(call_response)

        original_data_participant = original_data_participant.reset_index(drop=True)
        index_participant = 0
        participant_idiom = ''
        count_analysis_participant = 0
        for idx, row in original_data_participant.iterrows():
            if (str(row['E-mail']) == participant_id):
                index_participant = idx
                participant_idiom = str(row['Native Idiom'])
                count_analysis_participant = 1
        if (count_analysis_participant == 0):
            print("No experiment to analyze found")
            return 'Nothing to do. Product - Task time: {}!'.format(call_response)

        print("Downloading video and font... " + experiment_id)

        original_video_id = original_video_url.split('/')[-2]
        stream_video = download_file(original_video_id)
        with open(video_input, mode="wb") as f:
            f.write(stream_video)

        font_input = "Roboto-Bold.ttf"
        original_font_url = "" #change
        original_font_id = original_font_url.split('/')[-2]
        stream_font = download_file(original_font_id)
        with open(font_input, mode="wb") as f:
            f.write(stream_font)

        print("Extracting audio... " + experiment_id)

        audio_input = experiment_id + '.wav'
        (
          ffmpeg.input(video_input)
          .output(audio_input, ac=1,acodec='pcm_s16le',format='wav')
          .run(overwrite_output=True)
        )

        print("Working on the audio... " + experiment_id)
        audio_bucket_name = "" #change
        audio_blob_name = 'audio/'+audio_input
        upload_to_bucket(client_storage_api, audio_blob_name, audio_input, audio_bucket_name)

        print("Setting audio config... " + experiment_id)

        storage_client = storage.Client()
        bucket_name = "" #change
        blobs = storage_client.list_blobs(bucket_name)

        filter_dir = ""
        [blob.name for blob in blobs if filter_dir in blob.name ]

        print("Transcribing audio... " + experiment_id)

        project_id = "" #change
        gcs_uri_upload = "" + audio_blob_name #change to gs://name/
        gcs_uri_download = "" #change gs://name/another_name
        transcript, words_offset, full_transcript, audio_input_duration, full_results = transcribe_batch_gcs_input_inline_output_v4(client_speech_api, project_id, gcs_uri_upload, gcs_uri_download, audio_input, participant_idiom)

        print("Creating sentences... " + experiment_id)

        texts = ''
        for text in transcript:
            texts += text + " "

        sentences_transcription = sent_tokenize(texts)
        sentence_offset = {'sentence': [], 'start_offset': [], 'end_offset': [], 'sentence_sentiment': [], 'sentiment_magnitude': []}

        print("Creating subtitle file... " + experiment_id)

        # Create file and dataframte
        subtitles_sentence_offset = pd.DataFrame(sentence_offset)
        subtitle_output = experiment_id + ".srt"
        subtitles_sentence_offset = generate_srt(full_results, gcs_uri_upload, subtitle_output, subtitles_sentence_offset)

        # Analysing subtitle sentiments
        subtitles_sentence_sentiment = ''
        subtitles_sentence_magnitude = ''
        subtitles_sentiment_score = 0
        for idx_subtitle, row_subtitle in subtitles_sentence_offset.iterrows():
            if ((len(row_subtitle) > 0) and (row_subtitle['sentence'] != '')):
                subtitle = str(row_subtitle['sentence'])
                subtitles_sentiment_score, subtitles_sentiment_magnitude = analyze_sentiment(client_natural_language_api, subtitle, participant_idiom)
                subtitles_sentence_sentiment = 'Neutral'
                if (subtitles_sentiment_magnitude >= 0.25):
                    if (subtitles_sentiment_score > 0.25):
                        subtitles_sentence_sentiment = 'Positive'
                    if (subtitles_sentiment_score < -0.25):
                        subtitles_sentence_sentiment = 'Negative'
                subtitles_sentence_offset.loc[idx_subtitle, 'sentence_sentiment'] = subtitles_sentence_sentiment
                subtitles_sentence_offset.loc[idx_subtitle, 'sentiment_magnitude'] = subtitles_sentiment_magnitude

        print("AI SUS Similarity - Bard API... " + experiment_id)

        # SUS similarity
        ai_sentences_offset = pd.DataFrame(sentence_offset)
        ai_sus_sentences = ''
        ai_gemini_sus_sentences = ''
        ai_gemini_sus_sentences_filename = experiment_id + " - AI Gemini Subtitle.txt"
        ai_gemini_sus_sentences = get_sus_sentences_from_ai_gemini(full_transcript, participant_idiom, ai_gemini_sus_sentences_filename)
        ai_sus_sentences_filename = ai_gemini_sus_sentences_filename
        ai_sus_sentences = ai_gemini_sus_sentences

        print("AI SUS Similarity - Natural Language API... " + experiment_id)

        ai_sentences_offset = analyze_sentiment_document(client_natural_language_api, ai_sus_sentences, participant_idiom, ai_sentences_offset)

        print("Analyzing SUS similarity... " + experiment_id)

        if (participant_idiom == "Portuguese"):
            nlp = spacy.load("pt_core_news_lg")
            sentence_set1 = ["Eu acho que gostaria de usar esse sistema com frequência.", "Eu acho o sistema desnecessariamente complexo.", "Eu achei o sistema fácil de usar.", "Eu acho que precisaria de ajuda de uma pessoa com conhecimentos técnicos para usar o sistema.", "Eu acho que as várias funções do sistema estão muito bem integradas.", "Eu acho que o sistema apresenta muita inconsistência.", "Eu imagino que as pessoas aprenderão como usar esse sistema rapidamente.", "Eu achei o sistema atrapalhado de usar.", "Eu me senti confiante ao usar o sistema.", "Eu precisei aprender várias coisas novas antes de conseguir usar o sistema."]
        else:
            nlp = spacy.load("en_core_web_lg")
            sentence_set1 = ["I think that I would like to use this system frequently.", "I found the system unnecessarily complex.", "I thought the system was easy to use.", "I think that I would need the support of a technical person to be able to use this system.", "I found the various functions in this system were well-integrated.", "I thought there was too much inconsistency in this system.", "I would imagine that most people would learn to use this system very quickly.", "I found the system very cumbersome to use.", "I felt very confident using the system.", "I needed to learn a lot of things before I could get going with this system."]

        # Compare each sentence in the first set to each sentence in the second set
        question = 1
        sus_identified_data = {'question': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'score': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'similarity': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'sentence1': sentence_set1, 'sentence2': ['', '', '', '', '', '', '', '', '', ''], 'sentence1_sentiment': ['', '', '', '', '', '', '', '', '', ''], 'sentence2_sentiment': ['', '', '', '', '', '', '', '', '', '']}
        sus_identified = pd.DataFrame(data=sus_identified_data)
        for sent1 in sentence_set1:
          for idx_sentences_offset, row_sentences_offset in ai_sentences_offset.iterrows():
            sent2 = row_sentences_offset['sentence']
            sentiment2_magnitude = row_sentences_offset['sentiment_magnitude']
            sentence2_sentiment = row_sentences_offset['sentence_sentiment']
            similarity_score = 0
            similarity_score = compare_sentence_similarity(sent1, sent2, nlp)
            if (similarity_score >= 0.25):
                if (sentiment2_magnitude >= 0.25):
                    score = 3
                    sentence1_sentiment = 'Neutral'
                    if (sentence2_sentiment == 'Positive'):
                        if(question % 2 == 0):
                            sentence1_sentiment = 'Negative'
                            if (similarity_score >= 0.750):
                                score = 1
                            else:
                                score = 2
                        else:
                            sentence1_sentiment = 'Positive'
                            if (similarity_score >= 0.750):
                                score = 5
                            else:
                                score = 4
                    if (sentence2_sentiment == 'Negative'):
                        if(question % 2 == 0):
                            sentence1_sentiment = 'Negative'
                            if (similarity_score >= 0.750):
                                score = 5
                            else:
                                score = 4
                        else:
                            sentence1_sentiment = 'Positive'
                            if (similarity_score >= 0.750):
                                score = 1
                            else:
                                score = 2

                    if (sus_identified.loc[question-1]['similarity'] < similarity_score):
                        sus_identified.loc[sus_identified.question==question, 'score'] = score
                        sus_identified.loc[sus_identified.question==question, 'similarity'] = similarity_score
                        sus_identified.loc[sus_identified.question==question, 'sentence1'] = sent1
                        sus_identified.loc[sus_identified.question==question, 'sentence2'] = sent2
                        sus_identified.loc[sus_identified.question==question, 'sentence1_sentiment'] = sentence1_sentiment
                        sus_identified.loc[sus_identified.question==question, 'sentence2_sentiment'] = sentence2_sentiment
          question += 1

        print("Working on video... " + experiment_id)

        frame_number = 0
        evaluated_frames = 0

        sentiment = {
            'valence': [],
            'time': [],
            'instant_satisfaction': [],
            'instant_satisfaction_score': [],
            'instant_offset': 0
        }
        video_sentiment = pd.DataFrame(sentiment)

        message_line1 = ''
        message_line2 = ''
        message_line3 = ''
        message_line4 = ''
        message_line5 = ''
        message_line6 = ''
        client = client_vision_api

        videoName = video_input

        capture = cv2.VideoCapture(video_input)
        success, frame = capture.read()
        frames = 1

        video_width_frame = 1920
        ratio_frame = frame.shape[1] / frame.shape[0]
        video_height_frame = int(video_width_frame / ratio_frame)

        video_width = frame.shape[1]
        video_height = frame.shape[0]

        video_output = videoName + ' - Output.avi'
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        fps = capture.get(cv2.CAP_PROP_FPS)
        output_video = cv2.VideoWriter(video_output, fourcc, fps, (video_width, video_height))

        subtitles_sentence_offset_frame = subtitles_sentence_offset
        subtitles_sentence_offset_frame['fps'] = fps
        subtitles_sentence_offset_frame = subtitles_sentence_offset.eval('start_frame_offset = start_offset * fps ')
        subtitles_sentence_offset_frame = subtitles_sentence_offset_frame.eval('end_frame_offset = end_offset * fps ')
        subtitles_sentence_offset_frame.eval('start_frame_offset = start_frame_offset.round(0)', engine='python', inplace=True)
        subtitles_sentence_offset_frame.eval('end_frame_offset = end_frame_offset.round(0)', engine='python', inplace=True)
        subtitles_sentence_offset_frame = subtitles_sentence_offset_frame.astype({'start_frame_offset':'int','end_frame_offset':'int'})

        while success:
            image_to_draw = frame
            cv2_rgb = cv2.cvtColor(image_to_draw,cv2.COLOR_BGR2RGB)
            pil = Image.fromarray(cv2_rgb)
            draw = ImageDraw.Draw(pil)

            if (frame_number == 0):

                crop = pil
                crop = crop.crop((1435, 400, 1920, 675))

                base_width= 1920
                wpercent = (base_width / float(crop.size[0]))
                hsize = int((float(crop.size[1]) * float(wpercent)))
                crop_resized = crop.resize((base_width, hsize), Image.Resampling.LANCZOS)

                b = io.BytesIO()
                crop_resized.save(b, 'png')
                im_bytes = b.getvalue()
                content = im_bytes
                image = vision.Image(content=content)

                response = client.face_detection(image=image)
                faces = response.face_annotations
                evaluated_frames += 1

                if (len(faces) > 0):

                    face = faces[0]
                    emotion_intensity = vision.Likelihood.LIKELY
                    instant_affect = 'Neutral'
                    emotion_valence = 0
                    emotion_confidence = face.detection_confidence
                    valence = 'neutral'
                    if face.joy_likelihood > emotion_intensity:
                        emotion_intensity = face.joy_likelihood
                        instant_affect = 'Joy'
                        emotion_valence = 1
                        valence = 'positive'
                    if face.sorrow_likelihood > emotion_intensity:
                        emotion_intensity = face.sorrow_likelihood
                        instant_affect = 'Sorrow'
                        emotion_valence = -1
                        valence = 'negative'
                    if face.anger_likelihood > emotion_intensity:
                        emotion_intensity = face.anger_likelihood
                        instant_affect = 'Anger'
                        emotion_valence = -1
                        valence = 'negative'
                    if face.surprise_likelihood > emotion_intensity:
                        emotion_intensity = face.surprise_likelihood
                        instant_affect = 'Surprise'
                        emotion_valence = 0
                        valence = 'neutral'
                    if (face.joy_likelihood == face.sorrow_likelihood) and (face.sorrow_likelihood == face.anger_likelihood) and (face.anger_likelihood == face.surprise_likelihood):
                        instant_affect = 'Neutral'
                        emotion_valence = 0
                        valence = 'neutral'

                    # Transform emotion intensity in a number
                    intensity_score = 0
                    if ((emotion_intensity == vision.Likelihood.VERY_LIKELY)):
                            intensity_score = 1

                    # Identify concidence score
                    minutes = int(frames / (fps * 60))
                    seconds = int(int(frames % (fps * 60)) / fps)
                    total_seconds = int(frames / fps)
                    time = str(minutes) + "m" + str(seconds) + "s"
                    confidence_score = 0
                    if (emotion_confidence >= 0.5):
                        confidence_score = 1

                    instant_satisfaction_score = 0
                    instant_satisfaction_score = intensity_score * emotion_valence * confidence_score

                    instant_satisfaction = 'Neither satisfied nor dissatisfied'
                    if (instant_satisfaction_score == -1):
                            instant_satisfaction = 'Dissatisfied'
                    elif (instant_satisfaction_score == 1):
                            instant_satisfaction = 'Satisfied'

                    instant_message = ''

                    if ((valence == 'positive') or (valence == 'negative')):
                        video_sentiment.loc[len(video_sentiment.index)] = [valence, time, instant_satisfaction, instant_satisfaction_score, total_seconds]

                    if response.error.message:
                        raise Exception(
                            "{}\nFor more info on error messages, check: "
                            "https://cloud.google.com/apis/design/errors".format(response.error.message)
                        )

                    if (confidence_score == 0):
                        message_line1 = 'Wait for a better visualization.'
                        message_line2 = 'Low Confidence Score'
                        message_line3 = ''
                    else:

                        if len(instant_satisfaction) > 0:
                            message_line1 = "Satisfaction: " + instant_satisfaction
                        if len(instant_satisfaction) > 0:
                            message_line2 = "Affect (face): " +  instant_affect
                else:
                    message_line1 = 'Wait for a face detection.'
                    message_line2 = ''
                    message_line3 = ''

            message_line3 = ''
            message_line4 = ''
            message_line5 = ''
            mask1 = subtitles_sentence_offset_frame['start_frame_offset'].values <= frames
            row_subtitle_sentences_offset_frame = subtitles_sentence_offset_frame[mask1]
            row_subtitle_sentences_offset_frame.dropna()
            row_subtitle_sentences_offset_frame.reset_index(drop=True)
            if (len(row_subtitle_sentences_offset_frame) > 0):
                mask2 = row_subtitle_sentences_offset_frame['end_frame_offset'].values > frames
                row_subtitle_sentences_offset_frame = row_subtitle_sentences_offset_frame[mask2]
                row_subtitle_sentences_offset_frame.dropna()
            affect_text_message = ''
            row_subtitle_sentences_offset_frame.reset_index(drop=True)
            sentence_message = ''
            if (len(row_subtitle_sentences_offset_frame) > 0):
                affect_text_message = str(row_subtitle_sentences_offset_frame.iloc[0]['sentence_sentiment'])
                message_line3 = "Affect (speech): " + affect_text_message
                # Subtitles
                sentence_message = str(row_subtitle_sentences_offset_frame.iloc[0]['sentence'])
                words_message = word_tokenize(sentence_message)
                words=[word.lower() for word in words_message if word.isalpha()]
                message_line4 = ''
                message_line5 = ''
                if (len(words_message) >= 8):
                    size_message4 = math.ceil(len(words_message)/2)
                    if not (words_message[size_message4].isalpha()):
                        size_message4 += 1
                    size_message5 = len(words_message) - size_message4
                    size_message = len(words_message)
                    word_count = 1
                    for word_message in words_message:
                        if (word_count < size_message4):
                            if (words_message[word_count].isalpha()):
                                message_line4 += word_message + " "
                            else:
                                message_line4 += word_message
                        elif (word_count == size_message4):
                            message_line4 += word_message

                        else:
                            if (word_count < size_message):
                                if (words_message[word_count].isalpha()):
                                    message_line5 += word_message + " "
                                else:
                                    message_line5 += word_message
                            elif (word_count == size_message):
                                message_line5 += word_message
                        word_count += 1
                else:
                    message_line4 = sentence_message
                    message_line5 = ''

            if ((message_line1 != '') or (message_line2 != '') or (message_line3 != '') or (message_line4 != '') or (message_line5 != '')):
                alpha = 0.60
                x,y,w,h = round(video_width-660),15,650,70
                font = ImageFont.truetype(font_input, 28)

                shape1 = [(x, y), (x + w, y + h + 80)]
                draw.rectangle(shape1, fill ="#000000")
                draw.text((x + int(w/10),y + int(h/1.75)), message_line1, font=font)
                draw.text((x + int(w/10),y + int(h/1.75) + int(h/2)), message_line2, font=font)
                draw.text((x + int(w/10),y + int(h/1.75) + int(h/2) + int(h/2)), message_line3, font=font)

                if ((message_line4 != '') or (message_line5 != '')):
                    x1,y1,w1,h1 = round(video_width/4),30,800,15

                    shape2 = [(x1, video_height - 150), (x1 + w1, video_height - 50)]
                    draw.rectangle(shape2, fill ="#000000")
                    draw.text((x1 + int(w1/10), video_height - 130), message_line4, font=font)
                    draw.text((x1 + int(w1/10), video_height - 100), message_line5, font=font)

                overlay = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
                cv2.addWeighted(overlay, alpha, frame, 1 - alpha,0, frame)

            if (frame_number == round(1.5*fps) -1):
                frame_number = 0
            else:
                frame_number += 1

            output_video.write(frame)

            success, frame = capture.read()
            frames += 1

            if not success:
                break

        user_value_identified_score = 0
        user_value_identified = 'Neither satisfied nor dissatisfied'
        if (len(video_sentiment) > 0):
            user_value_identified_score = round(video_sentiment['instant_satisfaction_score'].sum() / len(video_sentiment.index),0)

            if (user_value_identified_score == -1):
                    user_value_identified = 'Dissatisfied'
            elif (user_value_identified_score == 1):
                    user_value_identified = 'Satisfied'

        print("Video Input: " + video_input)
        print("Video Output: " + video_output)
        print("Evaluated frames: " + str(evaluated_frames))
        print("Overall satisfaction: " + user_value_identified)
        print("Sentiment Peaks: ")
        print(video_sentiment)
        print("Geral Satisfaction: " + user_value_identified)
        print("Finish")
        output_video.release()
        cv2.destroyAllWindows()

        print("Find positive and negative user value satisfaction... " + experiment_id)

        video_sentiment_positive = video_sentiment.mask(video_sentiment['valence'] != "positive")
        video_sentiment_positive = video_sentiment_positive.dropna()

        video_sentiment_negative = video_sentiment.mask(video_sentiment['valence'] != "negative")
        video_sentiment_negative = video_sentiment_negative.dropna()

        sentiment_peak_positive_score = 0
        sentiment_peak_negative_score = 0
        sentiment_duration = {
            'valence': [],
            'time': [],
            'instant_satisfaction': [],
            'instant_satisfaction_score': [],
            'duration': []
        }

        # All positive and negative intervals
        all_sentiment_positive_duration_intervals = pd.DataFrame(sentiment_duration)
        all_sentiment_positive_duration = 0
        all_sentiment_peak_positive_score = 0

        all_sentiment_negative_duration_intervals = pd.DataFrame(sentiment_duration)
        all_sentiment_negative_duration = 0
        all_sentiment_peak_negative_score = 0

        video_sentiment_positive.reset_index(drop=True, inplace=True)
        for ind in video_sentiment_positive.index:
            if (video_sentiment_positive.iloc[ind]['instant_satisfaction_score'] != all_sentiment_peak_positive_score):
                all_sentiment_peak_positive_score = video_sentiment_positive.iloc[ind]['instant_satisfaction_score']
                all_sentiment_positive_duration = 1
            elif (video_sentiment_positive.iloc[ind]['instant_satisfaction_score'] == all_sentiment_peak_positive_score):
                if (((ind-1) >= 0) and ((video_sentiment_positive.iloc[ind]['instant_offset'] == video_sentiment_positive.iloc[ind-1]['instant_offset']) or (video_sentiment_positive.iloc[ind]['instant_offset'] == ((video_sentiment_positive.iloc[ind-1]['instant_offset'])+1)))):
                    all_sentiment_positive_duration += 1

            if (((ind+1) == len(video_sentiment_positive)) or (((ind+1) < len(video_sentiment_positive)) and (video_sentiment_positive.iloc[ind+1]['instant_satisfaction_score'] != all_sentiment_peak_positive_score)) or (((ind+1) < len(video_sentiment_positive)) and ((((video_sentiment_positive.iloc[ind]['instant_offset'])+1) < video_sentiment_positive.iloc[ind+1]['instant_offset']))) ):
                if (all_sentiment_positive_duration > 1):
                    ind_time = ind-all_sentiment_positive_duration+1
                else:
                    ind_time = ind
                all_sentiment_positive_duration_intervals.loc[len(all_sentiment_positive_duration_intervals.index)] = [video_sentiment_positive.iloc[ind]['valence'], video_sentiment_positive.iloc[ind_time]['time'], video_sentiment_positive.iloc[ind]['instant_satisfaction'], video_sentiment_positive.iloc[ind]['instant_satisfaction_score'], all_sentiment_positive_duration]
                all_sentiment_positive_duration = 1

        video_sentiment_negative.reset_index(drop=True, inplace=True)
        for ind in video_sentiment_negative.index:
            if (video_sentiment_negative.iloc[ind]['instant_satisfaction_score'] != all_sentiment_peak_negative_score):
                all_sentiment_peak_negative_score = video_sentiment_negative.iloc[ind]['instant_satisfaction_score']
                all_sentiment_negative_duration = 1
            elif (video_sentiment_negative.iloc[ind]['instant_satisfaction_score'] == all_sentiment_peak_negative_score):
                if (((ind-1) >= 0) and ((video_sentiment_negative.iloc[ind]['instant_offset'] == video_sentiment_negative.iloc[ind-1]['instant_offset']) or (video_sentiment_negative.iloc[ind]['instant_offset'] == ((video_sentiment_negative.iloc[ind-1]['instant_offset'])+1)))):
                    all_sentiment_negative_duration += 1

            if (((ind+1) == len(video_sentiment_negative)) or (((ind+1) < len(video_sentiment_negative)) and (video_sentiment_negative.iloc[ind+1]['instant_satisfaction_score'] != all_sentiment_peak_negative_score)) or (((ind+1) < len(video_sentiment_negative)) and ((((video_sentiment_negative.iloc[ind]['instant_offset'])+1) < video_sentiment_negative.iloc[ind+1]['instant_offset']))) ):
                if (all_sentiment_negative_duration > 1):
                    ind_time = ind-all_sentiment_negative_duration+1
                else:
                    ind_time = ind
                all_sentiment_negative_duration_intervals.loc[len(all_sentiment_negative_duration_intervals.index)] = [video_sentiment_negative.iloc[ind]['valence'], video_sentiment_negative.iloc[ind_time]['time'], video_sentiment_negative.iloc[ind]['instant_satisfaction'], video_sentiment_negative.iloc[ind]['instant_satisfaction_score'], all_sentiment_negative_duration]
                all_sentiment_negative_duration = 1


        # Peaks video satisfaction
        positive_sentiment_peak_identified = ''
        max_duration_positive_sentiment_peak_identified = 0
        max_instant_satisfaction_positive_sentiment_peak_identified = 0
        for index, row in all_sentiment_positive_duration_intervals.iterrows():
              if (max_instant_satisfaction_positive_sentiment_peak_identified < row['instant_satisfaction_score']):
                  max_instant_satisfaction_positive_sentiment_peak_identified = row['instant_satisfaction_score']
                  positive_sentiment_peak_identified = ''
                  max_duration_positive_sentiment_peak_identified = row['duration']
                  positive_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                  if (index+1 < len(all_sentiment_positive_duration_intervals)):
                      positive_sentiment_peak_identified += '\n'

              elif (max_instant_satisfaction_positive_sentiment_peak_identified == row['instant_satisfaction_score']):
                  if (max_duration_positive_sentiment_peak_identified < row['duration']):
                      positive_sentiment_peak_identified = ''
                      max_duration_positive_sentiment_peak_identified = row['duration']
                      positive_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                      if (index+1 < len(all_sentiment_positive_duration_intervals)):
                          positive_sentiment_peak_identified += '\n'
                  if (max_duration_positive_sentiment_peak_identified == row['duration']):
                      positive_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                      if (index+1 < len(all_sentiment_positive_duration_intervals)):
                          positive_sentiment_peak_identified += '\n'

        negative_sentiment_peak_identified = ''
        max_duration_negative_sentiment_peak_identified = 0
        max_instant_satisfaction_negative_sentiment_peak_identified = 0
        for index, row in all_sentiment_negative_duration_intervals.iterrows():
              if (max_instant_satisfaction_negative_sentiment_peak_identified > row['instant_satisfaction_score']):
                  max_instant_satisfaction_negative_sentiment_peak_identified = row['instant_satisfaction_score']
                  negative_sentiment_peak_identified = ''
                  max_duration_negative_sentiment_peak_identified = row['duration']
                  negative_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                  if (index+1 < len(all_sentiment_negative_duration_intervals)):
                      negative_sentiment_peak_identified += '\n'

              elif (max_instant_satisfaction_negative_sentiment_peak_identified == row['instant_satisfaction_score']):
                  if (max_duration_negative_sentiment_peak_identified > row['duration']):
                      negative_sentiment_peak_identified = ''
                      max_duration_negative_sentiment_peak_identified = row['duration']
                      negative_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                      if (index+1 < len(all_sentiment_negative_duration_intervals)):
                          negative_sentiment_peak_identified += '\n'
                  if (max_duration_negative_sentiment_peak_identified == row['duration']):
                      negative_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
                      if (index+1 < len(all_sentiment_negative_duration_intervals)):
                          negative_sentiment_peak_identified += '\n'

        # All video satisfaction
        all_positive_sentiment_peak_identified = ''
        for index, row in all_sentiment_positive_duration_intervals.iterrows():
              all_positive_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
              if (index+1 < len(all_sentiment_positive_duration_intervals)):
                  all_positive_sentiment_peak_identified += '\n'

        all_negative_sentiment_peak_identified = ''
        for index, row in all_sentiment_negative_duration_intervals.iterrows():
              all_negative_sentiment_peak_identified += row['instant_satisfaction'] + '(video) - ' + row['time']
              if (index+1 < len(all_sentiment_negative_duration_intervals)):
                  all_negative_sentiment_peak_identified += '\n'


        print("Find positive and negative affect... " + experiment_id)

        sentiment_sentences_offset = subtitles_sentence_offset.copy()
        sentiment_sentences_offset.drop(columns=['sentence', 'end_offset', 'fps'], inplace=True)

        # Affect Peaks
        # Identifying list of positive and negative offsets peaks
        positive_sentiment_sentences_offset = sentiment_sentences_offset.drop(sentiment_sentences_offset[sentiment_sentences_offset['sentence_sentiment'] == 'Neutral'].index)
        positive_sentiment_sentences_offset = positive_sentiment_sentences_offset.drop(positive_sentiment_sentences_offset[positive_sentiment_sentences_offset['sentence_sentiment'] == 'Negative'].index)

        positive_peak_sentiment_sentences_offset = pd.DataFrame(data=None, columns=positive_sentiment_sentences_offset.columns)
        max_positive_sentiment_magnitude = 0
        for idx, row in positive_sentiment_sentences_offset.iterrows():
            if (max_positive_sentiment_magnitude < row['sentiment_magnitude']):
                positive_peak_sentiment_sentences_offset = pd.DataFrame(data=None, columns=positive_sentiment_sentences_offset.columns)
                max_positive_sentiment_magnitude = row['sentiment_magnitude']
                positive_peak_sentiment_sentences_offset.loc[len(positive_peak_sentiment_sentences_offset.index)] = [row['start_offset'], row['sentence_sentiment'], row['sentiment_magnitude']]
            elif (max_positive_sentiment_magnitude == row['sentiment_magnitude']):
                positive_peak_sentiment_sentences_offset.loc[len(positive_peak_sentiment_sentences_offset.index)] = [row['start_offset'], row['sentence_sentiment'], row['sentiment_magnitude']]

        negative_sentiment_sentences_offset = sentiment_sentences_offset.drop(sentiment_sentences_offset[sentiment_sentences_offset['sentence_sentiment'] == 'Neutral'].index)
        negative_sentiment_sentences_offset = negative_sentiment_sentences_offset.drop(negative_sentiment_sentences_offset[negative_sentiment_sentences_offset['sentence_sentiment'] == 'Positive'].index)

        negative_peak_sentiment_sentences_offset = pd.DataFrame(data=None, columns=negative_sentiment_sentences_offset.columns)
        max_negative_sentiment_magnitude = 0
        for idx, row in negative_sentiment_sentences_offset.iterrows():
            if (max_negative_sentiment_magnitude < row['sentiment_magnitude']):
                negative_peak_sentiment_sentences_offset = pd.DataFrame(data=None, columns=negative_sentiment_sentences_offset.columns)
                max_negative_sentiment_magnitude = row['sentiment_magnitude']
                negative_peak_sentiment_sentences_offset.loc[len(negative_peak_sentiment_sentences_offset.index)] = [row['start_offset'], row['sentence_sentiment'], row['sentiment_magnitude']]
            elif (max_negative_sentiment_magnitude == row['sentiment_magnitude']):
                negative_peak_sentiment_sentences_offset.loc[len(negative_peak_sentiment_sentences_offset.index)] = [row['start_offset'], row['sentence_sentiment'], row['sentiment_magnitude']]


        # Create a string with the speech sentiment peaks
        idx_peak_offset_positive = 0
        if ((positive_sentiment_peak_identified != '') and (positive_sentiment_peak_identified[len(positive_sentiment_peak_identified)-1] != '\n')):
            positive_sentiment_peak_identified += '\n'
        for idx, row in positive_peak_sentiment_sentences_offset.iterrows():
            frames_peak = row['start_offset']
            minutes_peak = int(frames_peak / 60)
            seconds_peak = int(frames_peak % 60)
            time_peak = str(minutes_peak) + "m" + str(seconds_peak) + "s"
            positive_peak_offset = str(row['sentence_sentiment']) + " (speech) - " +  time_peak

            positive_sentiment_peak_identified += positive_peak_offset
            if ((idx_peak_offset_positive+1 < len(positive_peak_sentiment_sentences_offset))):
                positive_sentiment_peak_identified += '\n'
            if (positive_sentiment_peak_identified == ''):
                positive_sentiment_peak_identified = 'None'
            idx_peak_offset_positive += 1

        idx_peak_offset_negative = 0
        if ((negative_sentiment_peak_identified != '') and (negative_sentiment_peak_identified[len(negative_sentiment_peak_identified)-1] != '\n')):
            negative_sentiment_peak_identified += '\n'
        for idx, row in negative_peak_sentiment_sentences_offset.iterrows():
            frames_peak = row['start_offset']
            minutes_peak = int(frames_peak / 60)
            seconds_peak = int(frames_peak % 60)
            time_peak = str(minutes_peak) + "m" + str(seconds_peak) + "s"
            negative_peak_offset = str(row['sentence_sentiment']) + " (speech) - " +  time_peak

            negative_sentiment_peak_identified += negative_peak_offset
            if ((idx_peak_offset_negative+1 < len(negative_peak_sentiment_sentences_offset))):
                negative_sentiment_peak_identified += '\n'
            if (negative_sentiment_peak_identified == ''):
                negative_sentiment_peak_identified = 'None'
            idx_peak_offset_negative += 1

        # All speech sentiment
        idx_all_affect_positive = 0
        if (all_positive_sentiment_peak_identified == ''):
            all_positive_sentiment_peak_identified_tmp = ''
        else:
            all_positive_sentiment_peak_identified_tmp = all_positive_sentiment_peak_identified
            all_positive_sentiment_peak_identified_tmp += '\n'
        for idx_all_affect, row in positive_sentiment_sentences_offset.iterrows():
            frames_peak = row['start_offset']
            minutes_peak = int(frames_peak / 60)
            seconds_peak = int(frames_peak % 60)
            time_peak = str(minutes_peak) + "m" + str(seconds_peak) + "s"

            positive_peak_offset = str(row['sentence_sentiment']) + " (speech) - " +  time_peak
            all_positive_sentiment_peak_identified_tmp += positive_peak_offset

            if ((idx_all_affect_positive+1 < len(positive_sentiment_sentences_offset))):
                all_positive_sentiment_peak_identified_tmp += '\n'
            idx_all_affect_positive += 1
        if (all_positive_sentiment_peak_identified_tmp == ''):
            all_positive_sentiment_peak_identified_tmp = 'None'
        all_positive_sentiment_peak_identified = all_positive_sentiment_peak_identified_tmp

        idx_all_affect_negative = 0
        if (all_negative_sentiment_peak_identified == ''):
            all_negative_sentiment_peak_identified_tmp = ''
        else:
            all_negative_sentiment_peak_identified_tmp = all_negative_sentiment_peak_identified
            all_negative_sentiment_peak_identified_tmp += '\n'
        for idx_all_affect, row in negative_sentiment_sentences_offset.iterrows():
            frames_peak = row['start_offset']
            minutes_peak = int(frames_peak / 60)
            seconds_peak = int(frames_peak % 60)
            time_peak = str(minutes_peak) + "m" + str(seconds_peak) + "s"

            negative_peak_offset = str(row['sentence_sentiment']) + " (speech) - " +  time_peak
            all_negative_sentiment_peak_identified_tmp += negative_peak_offset

            if ((idx_all_affect_negative+1 < len(negative_sentiment_sentences_offset))):
                all_negative_sentiment_peak_identified_tmp += '\n'
            idx_all_affect_negative += 1
        if (all_negative_sentiment_peak_identified_tmp == ''):
            all_negative_sentiment_peak_identified_tmp = 'None'
        all_negative_sentiment_peak_identified = all_negative_sentiment_peak_identified_tmp

        print("Evaluating affect ... " + experiment_id)

        sentiment_positive_quantity = len(all_sentiment_positive_duration_intervals) + len(positive_sentiment_sentences_offset)
        sentiment_negative_quantity = len(all_sentiment_negative_duration_intervals) + len(negative_sentiment_sentences_offset)
        total_sentiment_quantity = sentiment_positive_quantity + sentiment_negative_quantity
        if (total_sentiment_quantity == 0):
            total_sentiment_quantity = 1
        positive_affect_identified = sentiment_positive_quantity / total_sentiment_quantity
        negative_affect_identified = sentiment_negative_quantity / total_sentiment_quantity
        affect_identified = ''
        if (positive_affect_identified >= 0.65):
            affect_identified = 'Positive'
        elif (negative_affect_identified >= 0.65):
            affect_identified = 'Negative'
        else:
            affect_identified = 'Neutral'

        print("Creating evaluated video and audio... " + experiment_id)

        audio_evaluated = mp.AudioFileClip(audio_input)
        video_evaluated = mp.VideoFileClip(video_output)
        final = video_evaluated.set_audio(audio_evaluated)
        evaluated_video_name = experiment_id + " - Evaluated.mp4"
        final.write_videofile(evaluated_video_name, threads = 16, fps=fps, preset='ultrafast', verbose=False, logger=None)

        print("Evaluating usability... " + experiment_id)

        sus_q1 = original_data_experiment['I think that I would like to use this system frequently'][index_experiment]
        match sus_q1:
            case 'Strongly agree':
                sus_q1_score = 5
            case 'Agree':
                sus_q1_score = 4
            case 'Neither agree nor disagree':
                sus_q1_score = 3
            case 'Disagree':
                sus_q1_score = 2
            case 'Strongly disagree':
                sus_q1_score = 1

        sus_q2 = original_data_experiment['I found the system unnecessarily complex'][index_experiment]
        match sus_q2:
            case 'Strongly agree':
                sus_q2_score = 5
            case 'Agree':
                sus_q2_score = 4
            case 'Neither agree nor disagree':
                sus_q2_score = 3
            case 'Disagree':
                sus_q2_score = 2
            case 'Strongly disagree':
                sus_q2_score = 1

        sus_q3 = original_data_experiment['I thought the system was easy to use'][index_experiment]
        match sus_q3:
            case 'Strongly agree':
                sus_q3_score = 5
            case 'Agree':
                sus_q3_score = 4
            case 'Neither agree nor disagree':
                sus_q3_score = 3
            case 'Disagree':
                sus_q3_score = 2
            case 'Strongly disagree':
                sus_q3_score = 1

        sus_q4 = original_data_experiment['I think that I would need the support of a technical person to be able to use this system'][index_experiment]
        match sus_q4:
            case 'Strongly agree':
                sus_q4_score = 5
            case 'Agree':
                sus_q4_score = 4
            case 'Neither agree nor disagree':
                sus_q4_score = 3
            case 'Disagree':
                sus_q4_score = 2
            case 'Strongly disagree':
                sus_q4_score = 1

        sus_q5 = original_data_experiment['I found the various functions in this system were well-integrated'][index_experiment]
        match sus_q5:
            case 'Strongly agree':
                sus_q5_score = 5
            case 'Agree':
                sus_q5_score = 4
            case 'Neither agree nor disagree':
                sus_q5_score = 3
            case 'Disagree':
                sus_q5_score = 2
            case 'Strongly disagree':
                sus_q5_score = 1

        sus_q6 = original_data_experiment['I thought there was too much inconsistency in this system'][index_experiment]
        match sus_q6:
            case 'Strongly agree':
                sus_q6_score = 5
            case 'Agree':
                sus_q6_score = 4
            case 'Neither agree nor disagree':
                sus_q6_score = 3
            case 'Disagree':
                sus_q6_score = 2
            case 'Strongly disagree':
                sus_q6_score = 1

        sus_q7 = original_data_experiment['I would imagine that most people would learn to use this system very quickly'][index_experiment]
        match sus_q7:
            case 'Strongly agree':
                sus_q7_score = 5
            case 'Agree':
                sus_q7_score = 4
            case 'Neither agree nor disagree':
                sus_q7_score = 3
            case 'Disagree':
                sus_q7_score = 2
            case 'Strongly disagree':
                sus_q7_score = 1

        sus_q8 = original_data_experiment['I found the system very cumbersome to use'][index_experiment]
        match sus_q8:
            case 'Strongly agree':
                sus_q8_score = 5
            case 'Agree':
                sus_q8_score = 4
            case 'Neither agree nor disagree':
                sus_q8_score = 3
            case 'Disagree':
                sus_q8_score = 2
            case 'Strongly disagree':
                sus_q8_score = 1

        sus_q9 = original_data_experiment['I felt very confident using the system'][index_experiment]
        match sus_q9:
            case 'Strongly agree':
                sus_q9_score = 5
            case 'Agree':
                sus_q9_score = 4
            case 'Neither agree nor disagree':
                sus_q9_score = 3
            case 'Disagree':
                sus_q9_score = 2
            case 'Strongly disagree':
                sus_q9_score = 1

        sus_q10 = original_data_experiment['I needed to learn a lot of things before I could get going with this system'][index_experiment]
        match sus_q10:
            case 'Strongly agree':
                sus_q10_score = 5
            case 'Agree':
                sus_q10_score = 4
            case 'Neither agree nor disagree':
                sus_q10_score = 3
            case 'Disagree':
                sus_q10_score = 2
            case 'Strongly disagree':
                sus_q10_score = 1

        sus_informed_score = 2.5 * ((sus_q1_score - 1) + (sus_q3_score - 1) + (sus_q5_score - 1) + (sus_q7_score - 1) + (sus_q9_score - 1) + (5 - sus_q2_score) + (5 - sus_q4_score) + (5 - sus_q6_score) + (5 - sus_q8_score) + (5 - sus_q10_score))
        usability_informed = ''

        if ((sus_informed_score >= 65)):
                usability_informed = 'Good'
        elif ((sus_informed_score >= 35) and (sus_informed_score < 65)):
                usability_informed = 'Neutral'
        else:
                usability_informed = 'Poor'

        sus_identified_score = 2.5 * ((sus_identified['score'][0] - 1) + (sus_identified['score'][2] - 1) + (sus_identified['score'][4] - 1) + (sus_identified['score'][6] - 1) + (sus_identified['score'][8] - 1) + (5 - sus_identified['score'][1]) + (5 - sus_identified['score'][3]) + (5 - sus_identified['score'][5]) + (5 - sus_identified['score'][7]) + (5 - sus_identified['score'][9]))
        usability_identified = ''

        if (sus_identified_score >= 65):
                usability_identified = 'Good'
        elif ((sus_identified_score >= 35) and (sus_identified_score < 65)):
                usability_identified = 'Neutral'
        else:
                usability_identified = 'Poor'

        print("Evaluating User Experience... " + experiment_id)
        user_experience_evaluation_score = 0
        match usability_identified:
            case 'Good':
                user_experience_evaluation_score += 1
            case 'Poor':
                user_experience_evaluation_score -= 1

        match affect_identified:
            case 'Positive':
                user_experience_evaluation_score += 1
            case 'Negative':
                user_experience_evaluation_score -= 1

        match user_value_identified:
            case 'Satisfied':
                user_experience_evaluation_score += 1
            case 'Dissatisfied':
                user_experience_evaluation_score -= 1

        user_experience_evaluation = ''
        if ((user_experience_evaluation_score >= 2)):
            user_experience_evaluation = 'Good'
        elif ((user_experience_evaluation_score >= -1) and (user_experience_evaluation_score < 2)):
            user_experience_evaluation = 'Neither good nor poor'
        else:
            user_experience_evaluation = 'Poor'

        print("Preparing to upload evaluated subtitle, audio and video... " + experiment_id)

        creds, _ = default()
        prefix = 'https://drive.google.com/file/d/'
        suffix = '/view?usp=drive_link'
        evaluated_video_filetype = 'video/mp4'
        evaluated_video_folder_id = "" #change
        if (evaluated_video_url != ''):
            evaluated_video_id = evaluated_video_url.split('/')[-2]
            evaluated_video_id = upload_basic(evaluated_video_name, evaluated_video_filetype, evaluated_video_folder_id, evaluated_video_id)
        else:
            evaluated_video_id = ''
            evaluated_video_id = upload_basic(evaluated_video_name, evaluated_video_filetype, evaluated_video_folder_id, evaluated_video_id)
            evaluated_video_url = prefix + evaluated_video_id + suffix

        evaluated_audio_filetype = 'audio/wav'
        evaluated_audio_folder_id = "" #change
        if (evaluated_audio_url != ''):
            evaluated_audio_id = evaluated_audio_url.split('/')[-2]
            evaluated_audio_id = upload_basic(audio_input, evaluated_audio_filetype, evaluated_audio_folder_id, evaluated_audio_id)
        else:
            evaluated_audio_id = ''
            evaluated_audio_id = upload_basic(audio_input, evaluated_audio_filetype, evaluated_audio_folder_id, evaluated_audio_id)
            evaluated_audio_url = prefix + evaluated_audio_id + suffix

        evaluated_subtitle_filetype = 'text/srt'
        evaluated_subtitle_folder_id = "" #change
        if (evaluated_subtitle_url != ''):
            evaluated_subtitle_id = evaluated_subtitle_url.split('/')[-2]
            evaluated_subtitle_id = upload_basic(subtitle_output, evaluated_subtitle_filetype, evaluated_subtitle_folder_id, evaluated_subtitle_id)
        else:
            evaluated_subtitle_id = ''
            evaluated_subtitle_id = upload_basic(subtitle_output, evaluated_subtitle_filetype, evaluated_subtitle_folder_id, evaluated_subtitle_id)
            evaluated_subtitle_url = prefix + evaluated_subtitle_id + suffix

        evaluated_ai_sus_sentences_filetype = 'text/srt'
        evaluated_ai_sus_sentences_folder_id = "" #change
        if (evaluated_ai_sus_sentences_url != ''):
            evaluated_ai_sus_sentences_id = evaluated_ai_sus_sentences_url.split('/')[-2]
            evaluated_ai_sus_sentences_id = upload_basic(ai_sus_sentences_filename, evaluated_ai_sus_sentences_filetype, evaluated_ai_sus_sentences_folder_id, evaluated_ai_sus_sentences_id)
        else:
            evaluated_ai_sus_sentences_id = ''
            evaluated_ai_sus_sentences_id = upload_basic(ai_sus_sentences_filename, evaluated_ai_sus_sentences_filetype, evaluated_ai_sus_sentences_folder_id, evaluated_ai_sus_sentences_id)
            evaluated_ai_sus_sentences_url = prefix + evaluated_ai_sus_sentences_id + suffix


        print("Sharing evaluated subtitle, audio, and video... " + experiment_id)
        creds, _ = default()
        share_file(
              real_file_id=evaluated_video_id,
              real_user=participant_id,
              real_domain="uxapp.com.br",
          )

        share_file(
              real_file_id=evaluated_audio_id,
              real_user=participant_id,
              real_domain="uxapp.com.br",
          )

        share_file(
            real_file_id=evaluated_subtitle_id,
            real_user=participant_id,
            real_domain="uxapp.com.br",
        )

        print("Deleting audio from bucket... " + experiment_id)

        delete_blob(audio_bucket_name, audio_blob_name)
        print("Preparing data to update... " + experiment_id)
        tz = pytz.timezone('America/Sao_Paulo')
        now = datetime.now(tz=tz)
        current_time = now.strftime("%H:%M:%S")
        job_end_time = current_time
        print("Current Time =", job_end_time)

        original_data_experiment.loc[int(index_experiment), 'Evaluated Link'] = evaluated_video_url
        original_data_experiment.loc[int(index_experiment), 'Evaluated Audio Link'] = evaluated_audio_url
        original_data_experiment.loc[int(index_experiment), 'Evaluated Subtitle Link'] = evaluated_subtitle_url
        original_data_experiment.loc[int(index_experiment), 'Evaluated AI SUS Sentences Link'] = evaluated_ai_sus_sentences_url
        original_data_experiment.loc[int(index_experiment), 'Positive Sentiment Peak Identified'] = positive_sentiment_peak_identified
        original_data_experiment.loc[int(index_experiment), 'Negative Sentiment Peak Identified'] = negative_sentiment_peak_identified
        original_data_experiment.loc[int(index_experiment), 'All Positive Sentiment Identified'] = all_positive_sentiment_peak_identified
        original_data_experiment.loc[int(index_experiment), 'All Negative Sentiment Identified'] = all_negative_sentiment_peak_identified
        original_data_experiment.loc[int(index_experiment), 'User Value Identified'] = user_value_identified
        original_data_experiment.loc[int(index_experiment), 'Usability Informed'] = usability_informed
        original_data_experiment.loc[int(index_experiment), 'Usability Identified'] = usability_identified
        original_data_experiment.loc[int(index_experiment), 'Usability Score Informed (SUS)'] = sus_informed_score
        original_data_experiment.loc[int(index_experiment), 'Usability Score Identified (SUS)'] = sus_identified_score
        original_data_experiment.loc[int(index_experiment), 'Affect Identified'] = affect_identified
        original_data_experiment.loc[int(index_experiment), 'User Experience Evaluation'] = user_experience_evaluation
        original_data_experiment.loc[int(index_experiment), 'Job Start'] = job_start_time
        original_data_experiment.loc[int(index_experiment), 'Job End'] = job_end_time
        original_data_experiment.loc[int(index_experiment), 'Status Script'] = 'DONE'
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q1'] = int(sus_identified['score'][0])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q2'] = int(sus_identified['score'][1])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q3'] = int(sus_identified['score'][2])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q4'] = int(sus_identified['score'][3])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q5'] = int(sus_identified['score'][4])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q6'] = int(sus_identified['score'][5])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q7'] = int(sus_identified['score'][6])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q8'] = int(sus_identified['score'][7])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q9'] = int(sus_identified['score'][8])
        original_data_experiment.loc[int(index_experiment), 'SUS Identified Q10'] = int(sus_identified['score'][9])

        values_update = []
        row_update = []

        row_update = original_data_experiment.iloc[int(index_experiment)].values.flatten().tolist()
        values_update.append(row_update)
        creds, _ = default()
        gsheet_update_values(SPREADSHEET_ID, RANGE_NAME, "USER_ENTERED", values_update,)

        print("End.")

    else:
        product = 'Nothing to do.'
        tasktime = "00/00/0000 00:00:00"

    call_response = product + ' - ' + tasktime
    return 'Product - Task time: {}!'.format(call_response)
